# Motor Loss Table Generation
This tool is used for motor loss table generation for GEN5SW

In [ ]:
import os
import glob
import numpy as np
import pandas as pd 
import math
import pandas_bokeh
pd.set_option("plotting.backend", "pandas_bokeh")
pandas_bokeh.output_notebook()

In [ ]:
def peak_2_rms(value_peak):
    value_rms = value_peak / math.sqrt(2)
    return value_rms

def rms_2_peak(value_rms):
    value_peak = value_rms * math.sqrt(2)
    return value_peak

def rpm_2_rads(speed_rpm):
    speed_rads = speed_rpm * 2 * math.pi 
    return speed_rads

def eff_pc(output_value, input_value):
   eff_pc = (output_value/input_value) * 100
   return eff_pc

In [ ]:
# Set directory of data
path = r'/Work/Motor Loss Table Gen/Data/220V/Q1'

# Join path and look for any .csv's
all_files = glob.glob(os.path.join(path, "*Signals*.csv"))

# Create dataframe from all_files csv's
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [ ]:
df.rename(columns = {
                    ' TesOp_B.L2m_TarTrq_IOP'                   :'T_Demanded',
                    ' tesOutputData.L2mTes_EstTrq.val_IOP'      :'T_Estimated',
                    'Transducer_Torque_IOP'                     :'T_Measured',
                    'Transducer_Speed_IOP'                      :'N_Measured',
                    ' tesInputData.L2mPosSpdArb_RotorSpd_IOP'   :'N_Unit_rpm',
                    ' sensvdcOutputData.L2mSensVdc_Vdc.val_IOP' :'V_DC',
                    ' sensidcOutputData.L2mSensIdc_Idc.val_IOP' :'I_DC',
                    ' posSpdArbInputData.L2mSensVmt_Vd_IOP'     :'Ud_RMS',
                    ' posSpdArbInputData.L2mSensVmt_Vq_IOP'     :'Uq_RMS',
                    ' tesOutputData.L2mTes_Id.val_IOP'          :'Id_Peak',
                    ' tesOutputData.L2mTes_Iq.val_IOP'          :'Iq_Peak',
                    'InverterEfficiency_IOP'                    :'Loss_Inv_Unit'
                    }, inplace = True)

df.drop(columns = {
                    ' TesOp_B.L2m_TarTrq_MCP',
                    ' tesOutputData.L2mTes_EstTrq.val_MCP',
                    'Transducer_Torque_MCP','Transducer_Speed_MCP',
                    ' tesInputData.L2mPosSpdArb_RotorSpd_MCP',
                    ' sensvdcOutputData.L2mSensVdc_Vdc.val_MCP',
                    ' sensidcOutputData.L2mSensIdc_Idc.val_MCP',
                    ' posSpdArbInputData.L2mSensVmt_Vd_MCP',
                    ' posSpdArbInputData.L2mSensVmt_Vq_MCP',
                    ' tesOutputData.L2mTes_Id.val_MCP',
                    ' tesOutputData.L2mTes_Iq.val_MCP',
                    'InverterEfficiency_MCP', 
                    'E2E Value',
                    'timestamp'
                    }, inplace=True)


In [ ]:
# Post process signals

# Mechanical
df["N_Measured_rads"]   = rpm_2_rads(df["N_Measured"])
df["P_Measured"]        = df["N_Measured_rads"] * df["T_Measured"]

# DC Link
df["P_DC"]              = df["V_DC"] * df["I_DC"]

# Phase Peak 
df["Ud_Peak"]           = rms_2_peak(df["Ud_RMS"]) 
df["Uq_Peak"]           = rms_2_peak(df["Uq_RMS"]) 
  
# Phase RMS
df["Id_RMS"]            = peak_2_rms(df["Id_Peak"]) 
df["Iq_RMS"]            = peak_2_rms(df["Iq_Peak"]) 
  
# Phase 
df["P_Ph"]              = (3/2) * ( ( df["Ud_Peak"] * df["Id_Peak"] ) + ( df["Uq_Peak"] * df["Iq_Peak"] ) ) 

# Losses
df["Loss_Inv_Real"]     = df["P_DC"] - df["P_Ph"]
df["Loss_Mtr"]          = df["P_Measured"] - df["P_Ph"]
df["Loss_Mtr_Comp"]     = df["Loss_Mtr"] - df["Loss_Inv_Unit"]
df["Loss_Sys"]          = df["Loss_Inv_Real"] - df["Loss_Mtr"]


In [ ]:
# Plot test points
# could be faster by rounding or finding unique values?
df.plot.scatter(
                x       = "N_Measured",
                y       = "T_Demanded",
                c       = 'black',
                xticks  = np.arange(min(df["N_Measured"]), max(df["N_Measured"]), 1000),
                yticks  = np.arange(min(df["T_Demanded"]), max(df["T_Demanded"]), 20),
                title   = 'Tested Points',
                xlabel  = 'Speed (rpm)',
                ylabel  = 'Torque (Nm)',
                marker  = 'x'
                )

In [ ]:
# Plot Losses
df.plot.line(
            y       = ["Loss_Inv_Real","Loss_Inv_Unit","Loss_Mtr","Loss_Mtr_Comp"],
            title   = "Losses (W)",
            xlabel  = "Samples (n)",
            ylabel  = "Torque (Nm)"
            )

In [ ]:
# Plot Powers
df.plot.line(
            y       = ["P_DC","P_Ph","P_Measured"],
            title   = "Powers (W)",
            xlabel  = "Samples (n)",
            ylabel  = "Powers (W)"
            )

In [ ]:

Speed_Threshold = 0
Torque_Threshold = 0

# Determine operating quadrant
df["Operating_Quadrant"] = 'VEH_STP'
df["Operating_Quadrant"] = np.where((df["N_Measured"] > Speed_Threshold) & (df["T_Measured"] > Torque_Threshold), 'DRV_FWD', df["Operating_Quadrant"])
df["Operating_Quadrant"] = np.where((df["N_Measured"] > Speed_Threshold) & (df["T_Measured"] < Torque_Threshold), 'BRK_FWD', df["Operating_Quadrant"])
df["Operating_Quadrant"] = np.where((df["N_Measured"] < Speed_Threshold) & (df["T_Measured"] < Torque_Threshold), 'DRV_REV', df["Operating_Quadrant"])
df["Operating_Quadrant"] = np.where((df["N_Measured"] < Speed_Threshold) & (df["T_Measured"] > Torque_Threshold), 'BRK_REV', df["Operating_Quadrant"])

# Efficiency
df["Eff_Inv"] = '0'
np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') , eff_pc(df["P_Ph"], df["P_DC"]), df["Eff_Inv"] )
np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') , eff_pc(df["P_DC"], df["P_Ph"]), df["Eff_Inv"] )

df["Eff_Mtr"] = '0'
np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') , eff_pc(df["P_Measured"], df["P_Ph"]), df["Eff_Mtr"] )
np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') , eff_pc(df["P_Ph"], df["P_Measured"]), df["Eff_Mtr"] )

df["Eff_Sys"] = '0'
np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') , eff_pc(df["P_Measured"], df["P_DC"]), df["Eff_Sys"] )
np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') , eff_pc(df["P_DC"], df["P_Measured"]), df["Eff_Sys"] )

In [ ]:
# Transient Removal
Dwell_Period = 500

df["Step_Change"]   = '0'
df["Step_Change"]   = df["T_Demanded"].diff()

Step_index = (df.index[df['Step_Change'] != 0])
Stop_index = Step_index + Dwell_Period

delete_slice = [0]

for x in range(len(Step_index)):
    temp_slice = np.arange( Step_index[x], Stop_index[x], 1 )
    delete_slice.extend(temp_slice) 

df.drop(delete_slice)
